Este notebook levanta la carpeta data que se encuentra en el mismo directorio que el propio notebook y a partir de ella genera una csv y procesa los archivos de audio encontrados para generar espectrogramas de mell

In [9]:
import os
import torch
import torchaudio
import pandas as pd
import numpy as np
from tqdm import tqdm

In [10]:
#parametros para los calculos del mell
TARGET_SR = 16000  # Normalmente el audio se sule usar a 16k aunque encontre papers que trabajan a 22050 o a 22k (ver V2)
N_FFT = 512  #muestras de la fft
W_LEN = 400  # Numero de muestras para la ventan de la  fft (seg_de_ventan *sr) 
H_LEN = 160, # paso de la ventana entre una fft y la siguiente (paso * sr)
N_MELS = 26
#se suele usar una ventana de haning

#creo una bariable root la cual se puede cambiar si se quiere trabajar con otro directorio,
# pero a priori se deja en '.' para trabajar desde el directorio local del proyecto
root = '../data'
savePath = os.path.join(root,"..","csv")
dsName = "metaData.csv"

os.listdir(root)

['audio', 'NOISE', 'NOISE_16K', 'RIR', 'RIR_16K']

In [11]:
#En esta celda se recorre todo el arbol de carpetas para obtener todos los meta datos de las canciones y los path a sus correspondientes archivos de audio
add_instrumental = True

res = []
for genero in tqdm(os.listdir(root + '/audio')):
  if genero.endswith('.csv'):
    continue
  for artista in os.listdir(root + '/audio/' + genero):
    for album in os.listdir(root + '/audio/' + genero + '/' + artista):
      if genero == 'instrumental':
        if add_instrumental == True :
          if album.endswith((".mp3", ".wav", ".flac")):
            songName = os.path.splitext(album)[0]
            localPath = '/audio/' + genero + '/' + artista
            res.append([genero, artista, genero ,songName, localPath, localPath + '/' + album])
      else:
        for song in os.listdir(root + '/audio/' + genero + '/' + artista + '/' + album):
          if song.endswith((".mp3", ".wav", ".flac")):
            songName = os.path.splitext(song)[0]
            localPath = '/audio/' + genero + '/' + artista + '/' + album
            res.append([genero, artista, album, songName, localPath, localPath+'/'+song])

#creo un archivo csv con los metadatos por si se quiere revisar y debugear
metaData = pd.DataFrame(res, columns=['MFI', 'artist', 'album', 'song', 'localPath', 'fileTrak'])
#if (not os.path.isdir('csv')):
#  os.makedirs('csv')
metaData.to_csv(os.path.join(savePath,dsName), index = False)

100%|██████████| 3/3 [00:00<00:00, 162.10it/s]


In [12]:
metaData.set_index('fileTrak', inplace=True)

In [13]:
errors = []
for f in tqdm(metaData.index):
    try:
        _signal, _rate = torchaudio.load(root+f)
        metaData.at[f, 'duracion'] = _signal.shape[1]/_rate
    except:
        errors.append(f)
        print('Error:', f)
    

100%|██████████| 1431/1431 [06:37<00:00,  3.60it/s]


In [14]:
#metaData.drop(errors, inplace = True)
metaData.to_csv(os.path.join(savePath,dsName), index = True)

In [15]:
print(errors)

[]
